In [0]:
import os

!pip install --upgrade tables
!pip install eli5

os.kill(os.getpid(), 9)

In [0]:
from google.colab import drive
ROOT_DIR = "/content/drive"
drive.mount(ROOT_DIR)

In [0]:
MATRIX_TWO_PATH = ROOT_DIR + "/My Drive/Colab Notebooks/matrix/matrix_two"
GITHUB_TOKEN = "47164c8eff933a6515202fa3deda6cc20cb5ebff"
GITHUB_REPO_URL = "https://{0}@github.com/njdarda/dw_matrix_cars.git".format(GITHUB_TOKEN)

import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [0]:
cd "{MATRIX_TWO_PATH}/dw_matrix_cars/"

## Importing data

In [0]:
df = pd.read_hdf('data/car.h5')

In [5]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

## Dummy model

In [6]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [0]:
df = df[df['price_currency'] == 'PLN']

In [8]:
df['price_currency'].value_counts()
df.shape

(106290, 155)

In [9]:
feats = ['car_id']
X = df[feats].values
y = df['price_value'].values

model = DummyRegressor()
model.fit(X,y)
y_pred = model.predict(X)

mae(y, y_pred)

39456.885927541574

## Features

In [0]:
suffix_cat = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  factorized_values = df[feat].factorize()[0]
  if suffix_cat in feat:
    df[feat] = factorized_values
  else:
    df[feat + suffix_cat] = factorized_values

In [11]:
cat_feats = [x for x in df.columns if suffix_cat in x]
cat_feats = [x for x in cat_feats if 'price' not in x]

len(cat_feats)

151

In [14]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y , cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368324

In [15]:
model_2 = DecisionTreeRegressor(max_depth=5)
model_2.fit(X, y)

imp = PermutationImportance(model_2, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2533 ± 0.0040,param_napęd__cat
0.2008 ± 0.0047,param_faktura-vat__cat
0.1943 ± 0.0088,param_stan__cat
0.1423 ± 0.0086,param_rok-produkcji__cat
0.0629 ± 0.0047,param_moc__cat
0.0424 ± 0.0014,feature_kamera-cofania__cat
0.0412 ± 0.0008,param_skrzynia-biegów__cat
0.0286 ± 0.0037,param_marka-pojazdu__cat
0.0191 ± 0.0022,param_pojemność-skokowa__cat
0.0163 ± 0.0005,feature_bluetooth__cat
